In [1]:
import os
import sys
module_path = os.path.abspath(os.path.join('../../src'))
if module_path not in sys.path:
    sys.path.append(module_path)
    
import numpy as np
from utils.model import load_pool, load_lenet
from utils.file import load_from_json
from utils.metrics import error_rate, get_corrections
from models.athena import Ensemble, ENSEMBLE_STRATEGY

Using TensorFlow backend.
/home/meng/anaconda3/envs/pytorch/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/meng/anaconda3/envs/pytorch/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/meng/anaconda3/envs/pytorch/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/meng/anaconda3/en

In [2]:
# global variables
CONFIG_ROOT = "../../src/configs/"

## Configure your candidate pools

You can use individual json files for each type of models. The configuration should be similar to ``"model-mnist.json"``. 

Or, you can use a single json file for all types of models (the approach we used for this example) --- ``"hybrid-mnist.json"``.

Sample configuration for specific type of model:

e.g., example for ``cnn`` models:
```json
"cnn": {
    "dir": "../../models/cnn/",
    "um_file": "model-mnist-cnn-clean.h5",
    "wd_prefix": "model-mnist-cnn-",
    "wd_postfix": ".h5",
    "architecture": "cnn"
}
```

----
* python example: `tutorials/eval_model.py`
* main api: `tutorials.eval_model.evaluate_hybrid`

In [3]:
# copy from tutorials/eval_model.py
def evaluate_hybrid(trans_configs, model_configs,
                    data_configs, save=False, output_dir=None):
    """
    Apply transformation(s) on images.
    :param trans_configs: dictionary. The collection of the parameterized transformations to test.
        in the form of
        { configsx: {
            param: value,
            }
        }
        The key of a configuration is 'configs'x, where 'x' is the id of corresponding weak defense.
    :param model_configs:  dictionary. Defines model related information.
        Such as, location, the undefended model, the file format, etc.
    :param data_configs: dictionary. Defines data related information.
        Such as, location, the file for the true labels, the file for the benign samples,
        the files for the adversarial examples, etc.
    :param save: boolean. Save the transformed sample or not.
    :param output_dir: path or str. The location to store the transformed samples.
        It cannot be None when save is True.
    :return:
    """
    # Load the baseline defense (PGD-ADT model)
    baseline = load_lenet(file=model_configs.get('pgd_trained'), trans_configs=None,
                                  use_logits=False, wrap=False)

    # get the CNN undefended model (UM)
    # Since the uploaded AEs were generated for the CNN UM,
    # collect the corrections based on the CNN UM.
    cnn_configs = model_configs.get('cnn')
    file = os.path.join(cnn_configs.get('dir'), cnn_configs.get('um_file'))
    undefended = load_lenet(file=file,
                            trans_configs=trans_configs.get('configs0'),
                            wrap=True)
    print(">>> um:", type(undefended))

    # load CNN weak defenses into a pool
    # tiny pool: 3 weak defenses
    cnn_pool, _ = load_pool(trans_configs=trans_configs,
                            model_configs=cnn_configs,
                            active_list=True,
                            wrap=True)

    cnns = list(cnn_pool.values())
    print(">>> CNNs:", type(cnns), type(cnns[0]))

    # load SVM weak defenses into a pool
    # tiny pool: 3 weak defenses
    svm_configs = model_configs.get('svm')
    svm_pool, _ = load_pool(trans_configs=trans_configs,
                            model_configs=svm_configs,
                            active_list=True,
                            wrap=True)

    svms = list(svm_pool.values())
    print(">>> SVMs:", type(svms), type(svms[0]))

    # ----------------------
    # Hybrid
    # ----------------------
    """
    There are multiple ways to select weak defenses for your ensemble.
    I shew you a naive strategy here.
    Try to come up your own strategy, so you will create several variants of
    hybrid ensembles. Evaluate all your ensemble variants.
    
    For examlpe, here is a naive strategy to build N hybrid variants:
    1. fix the number of total weak defenses in the hybrid ensemble. (e.g., 50)
    2. build a hybrid ensemble, starting from all CNN weak defenses. (50 cnns, 0 svms) -- h1
    3. build your next hybrid ensemble by replacing part of the CNN weak defenses with SVM weak defenses.
       For examples, replace 10%, then 20%, 30%, etc. of the weak defenses with SVMs,
       until you get a hybrid consisting only SVMs. 
       (45 cnns, 5 svms) -- h2; 
       (40 cnns, 10 svms) -- h3; 
       ...; 
       (0 cnns, 50 svms) -- hN.
    4. So, you will have 11 ensemble variants, varying the ratio between two types of models.
    """
    
    # Select weak defenses for your ensemble.
    # In this example, we build a hybrid ensemble consisting of 3 cnn models and 3 svm models
    wds = cnns
    wds.extend(svms)
    # create the ensemble
    ensemble = Ensemble(classifiers=wds,
                        strategy=ENSEMBLE_STRATEGY.AVEP.value)

    # -----------------------------
    # Evaluate models
    # -----------------------------
    # load the benign samples
    bs_file = os.path.join(data_configs.get('dir'), data_configs.get('bs_file'))
    x_bs = np.load(bs_file)
    img_rows, img_cols = x_bs.shape[1], x_bs.shape[2]

    # load the corresponding true labels
    label_file = os.path.join(data_configs.get('dir'), data_configs.get('label_file'))
    labels = np.load(label_file)

    # get indices of benign samples that are correctly classified by the targeted model
    print(">>> Evaluating UM on [{}], it may take a while...".format(bs_file))
    pred_bs = undefended.predict(x_bs)
    corrections = get_corrections(y_pred=pred_bs, y_true=labels)

    # Evaluate AEs.
    results = {}
    ae_list = data_configs.get('ae_files')
    ae_file = os.path.join(data_configs.get('dir'), ae_list[4])
    x_adv = np.load(ae_file)

    # evaluate the undefended model on the AE
    print(">>> Evaluating UM on [{}], it may take a while...".format(ae_file))
    pred_adv_um = undefended.predict(x_adv)
    err_um = error_rate(y_pred=pred_adv_um, y_true=labels, correct_on_bs=corrections)
    # track the result
    results['UM'] = err_um

    # evaluate the ensemble on the AE
    # if you have multiple ensemble variants, you need to perform this evaluation
    # for each of the ensembles.
    print(">>> Evaluating ensemble on [{}], it may take a while...".format(ae_file))
    pred_adv_ens = ensemble.predict(x_adv)
    err_ens = error_rate(y_pred=pred_adv_ens, y_true=labels, correct_on_bs=corrections)
    # track the result
    # you may want to use a different key here, in order to distinguish the ensemble variants.
    results['Ensemble'] = err_ens

    # evaluate the baseline on the AE
    print(">>> Evaluating baseline model on [{}], it may take a while...".format(ae_file))
    pred_adv_bl = baseline.predict(x_adv)
    err_bl = error_rate(y_pred=pred_adv_bl, y_true=labels, correct_on_bs=corrections)
    # track the result
    results['PGD-ADT'] = err_bl

    # TODO: collect and dump the evaluation results to file(s) such that you can analyze them later.
    print(">>> Evaluations on [{}]:\n{}".format(ae_file, results))


In [4]:
# load experiment configurations
trans_configs = load_from_json("../../src/configs/demo/athena-mnist.json")
model_configs = load_from_json("../../src/configs/demo/hybrid-mnist.json")
data_configs = load_from_json("../../src/configs/demo/data-mnist.json")

output_dir = "../../results"

# -------- Evaluate HYBRID ATHENA -------------
evaluate_hybrid(trans_configs=trans_configs,
                model_configs=model_configs,
                data_configs=data_configs,
                save=False,
                output_dir=output_dir)

>>> Loading model [../../models/baseline/advTrained-mnist-adtC.h5]...
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.
>>> Loading model [../../models/cnn/model-mnist-cnn-clean.h5]...
>>> um: <class 'models.keraswrapper.WeakDefense'>
>>> Loading model [../../models/cnn/model-mnist-cnn-shift_bottom_left.h5]...
>>> Loading model [../../models/cnn/model-mnist-cnn-affine_both_stretch.h5]...
>>> Loading model [../../models/cnn/model-mnist-cnn-cartoon_mean_type3.h5]...
>>> Loaded 3 models.
>>> CNNs: <class 'list'> <class 'models.keraswrapper.WeakDefense'>
>>> Loading model [../../models/svm/model-mnist-svm-shift_bottom_left.pkl]...
>>> Loading model [../../models/svm/model-mnist-svm-affine_both_stretch.pkl]...
>>> Loading model [../../models/svm/model-mnist-svm-cartoon_mean_type3.pkl]...
>>> Loaded 3 models.
